<a href="https://colab.research.google.com/github/gagan3012/rap-writer/blob/master/GPTModel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q kaggle
!mkdir ~/.kaggle
!echo '{"username":"gagan2000","key":"73e5d9db85677b98fe016b25873727ec"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d rikdifos/rap-lyrics -p /content

  0% 0.00/2.57M [00:00<?, ?B/s]
100% 2.57M/2.57M [00:00<00:00, 84.9MB/s]


In [3]:
!unzip /content/rap-lyrics.zip -d /content/rap

Archive:  /content/rap-lyrics.zip
  inflating: /content/rap/ASAP Ant_lyrics.txt  
  inflating: /content/rap/ASAP Rocky_lyrics.txt  
  inflating: /content/rap/Action Bronson_lyrics.txt  
  inflating: /content/rap/Andre 3000_lyrics.txt  
  inflating: /content/rap/Bas_lyrics.txt  
  inflating: /content/rap/Big L_lyrics.txt  
  inflating: /content/rap/Chance The Rapper_lyrics.txt  
  inflating: /content/rap/Childish Gambino_lyrics.txt  
  inflating: /content/rap/Common_lyrics.txt  
  inflating: /content/rap/CunninLynguists_lyrics.txt  
  inflating: /content/rap/Deniro Farrar_lyrics.txt  
  inflating: /content/rap/Drake_lyrics.txt  
  inflating: /content/rap/Earl Sweatshirt_lyrics.txt  
  inflating: /content/rap/Eazy-E_lyrics.txt  
  inflating: /content/rap/Eminem_lyrics.txt  
  inflating: /content/rap/Ice Cube_lyrics.txt  
  inflating: /content/rap/Immortal Technique_lyrics.txt  
  inflating: /content/rap/Isaiah Rashad_lyrics.txt  
  inflating: /content/rap/J Cole_lyrics.txt  
  inflating:

In [4]:
!pip install datasets
!git clone https://github.com/huggingface/transformers
!pip install transformers

     |████████████████████████████████| 184kB 18.1MB/s 
     |████████████████████████████████| 112kB 44.8MB/s 
     |████████████████████████████████| 245kB 55.6MB/s 
Cloning into 'transformers'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 64820 (delta 16), reused 2 (delta 0), pack-reused 64779
Receiving objects: 100% (64820/64820), 48.74 MiB | 29.55 MiB/s, done.
Resolving deltas: 100% (45939/45939), done.
     |████████████████████████████████| 1.9MB 21.7MB/s 
     |████████████████████████████████| 890kB 54.8MB/s 
     |████████████████████████████████| 3.2MB 49.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a23a57f602c1037b91ff9e6a880e43b40736eb7265d19f6ad955c54f9cd64d61
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import glob

In [6]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles  

In [7]:
filenames = getListOfFiles('/content/rap')
with open('/content/raplyrics.txt', 'w') as outfile:
    for fname in filenames:
        print(fname)
        with open(fname, encoding='utf-8',errors='ignore') as infile:
            for line in infile:
                outfile.write(line)

/content/rap/Eminem_lyrics.txt
/content/rap/Tupac1_lyrics.txt
/content/rap/Drake_lyrics.txt
/content/rap/Big L_lyrics.txt
/content/rap/Kanye West_lyrics.txt
/content/rap/The Notorious BIG_lyrics.txt
/content/rap/Eazy-E_lyrics.txt
/content/rap/Royce Da 59_lyrics.txt
/content/rap/Tupac2_lyrics.txt
/content/rap/Earl Sweatshirt_lyrics.txt
/content/rap/Logic_lyrics.txt
/content/rap/NF_lyrics.txt
/content/rap/Childish Gambino_lyrics.txt
/content/rap/Jay-z_lyrics.txt
/content/rap/Chance The Rapper_lyrics.txt
/content/rap/Immortal Technique_lyrics.txt
/content/rap/Joey Badass_lyrics.txt
/content/rap/ASAP Rocky_lyrics.txt
/content/rap/Ice Cube_lyrics.txt
/content/rap/Tyler The Creator_lyrics.txt
/content/rap/Nas_lyrics.txt
/content/rap/Scarface_lyrics.txt
/content/rap/Andre 3000_lyrics.txt
/content/rap/Kendrick Lamar_lyrics.txt
/content/rap/Action Bronson_lyrics.txt
/content/rap/Mac Miller_lyrics.txt
/content/rap/Montana of 300_lyrics.txt
/content/rap/J Cole_lyrics.txt
/content/rap/Lupe Fiasco_

In [8]:
with open('/content/raplyrics.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print(len(train))
print(len(eval))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

167933
18660


In [9]:
import os
os.chdir("/content/transformers/examples/")
os.chdir("./language-modeling")

In [10]:
!python run_clm.py \
--model_type gpt2 \
--model_name_or_path gpt2 \
--train_file "/content/train_tmp.txt" \
--do_train \
--validation_file "/content/eval_tmp.txt" \
--do_eval \
--per_device_train_batch_size 1 \
--per_device_eval_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="/content/model"

2021-02-28 18:30:50.920376: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/28/2021 18:30:52 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
02/28/2021 18:30:52 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/content/model, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Feb28_18-30-51_43334c911c08, logging_first_step=False, logging_steps=500, save_steps=-1, save_total_limit=No